In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('..', '..')))
from src import *
from itertools import product

import numpy as np
from matplotlib import pyplot as plt

In [2]:
moments = np.array([-2, -1, 2, 3, 4])
ps = np.array([0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15])
system_sizes = np.array([100, 200, 400, 800, 1600, 3200, 6400])
realizations = 10

# push to larger sizes

def produce_data(ps, system_sizes, moments, realizations, bins = 50):
    widths = [L // bins for L in system_sizes]
    fractal_dimension = np.zeros((len(ps), len(system_sizes), len(moments)))
    params = product(enumerate(ps), enumerate(system_sizes), enumerate(moments))
    for (i, p), (j, L), (k, m) in params:
        dimension_data_reduced = 0
        for _ in range(realizations):
            ness_correlation = C_NESS(0, 0, A(L, p))
            _, eigenvectors = np.linalg.eig(ness_correlation)
            moment_data = calculate_moments(eigenvectors.T, widths[j], m)
            dimension_data = generalized_fractal_dimension(moment_data, widths[j], m, L)
            dimension_data_reduced += np.mean(list(dimension_data))
        fractal_dimension[i,j,k] = dimension_data_reduced / realizations
    return fractal_dimension

data = produce_data(ps, system_sizes, moments, realizations)

In [3]:
np.save("../../data/multifractal_data.npy", data)